In [1]:
import json
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_community.tools.tavily_search import TavilySearchResults

from langgraph.graph import StateGraph, MessagesState, START, END, MessageGraph
from IPython.display import display, Image
from typing import TypedDict, Annotated, Literal
from pydantic import BaseModel, Field
import operator
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode

load_dotenv("../.env")
llm = ChatGroq(
    # model="llama3-groq-70b-8192-tool-use-preview",
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)
#embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

memory = MemorySaver()
config = {"configurable": {"thread_id": 1}}

In [2]:
def entry(input: list[HumanMessage]):
    return input

def human(input: list[HumanMessage]):
    input[0].content += " is not Amazing!"
    return input

def ai(input: list[HumanMessage]):
    input[0].content += " is Amazing!"
    return input

def finish(input: list[HumanMessage]):
    input[0].content += " always!"
    return input

def router_node2_or_3(input: list[HumanMessage]):
    if input[0].content == "human":
        return "human_node"
    else:
        return "ai_node"

In [3]:
graph = MessageGraph()

node1_id = "entry"
node2_id = "human"
node3_id = "ai"
node4_id = "finish"

graph.add_node(node1_id, entry)
graph.add_node(node2_id, human)
graph.add_node(node3_id, ai)
graph.add_node(node4_id, finish)

graph.add_conditional_edges(
    node1_id,
    router_node2_or_3,
    {"human_node": node2_id, "ai_node": node3_id}
)

graph.add_edge(node2_id, node4_id)
graph.add_edge(node3_id, node4_id)
graph.add_edge(node4_id, END)
graph.set_entry_point(node1_id)

app = graph.compile()

In [4]:
def save_graph(runnable_graph, file_path):
    png_out = runnable_graph.get_graph().draw_mermaid_png()
    with open(file_path, "wb") as f:
        f.write(png_out)

In [5]:
save_graph(app,"second_run.png")

In [6]:
print(app.invoke("Ai")[0].content)

Ai is Amazing! always!


In [7]:
print(app.invoke("human")[0].content)

human is not Amazing! always!
